In [ ]:
Dataset- https://www.kaggle.com/competitions/fake-news/data

In [199]:
import pandas as pd

In [200]:
df=pd.read_csv('news_train.csv')

In [201]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [202]:
df=df.dropna()

In [203]:
X=df.drop('label',axis=1)

In [204]:
y=df['label']

In [205]:
X.shape

(18285, 4)

In [206]:
y.shape

(18285,)

In [207]:
import tensorflow as tf

In [208]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [209]:
voc_size=5000

In [210]:
#Onehot representation
messages=X.copy()
messages.reset_index(inplace=True)
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [211]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [212]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maddy16/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [213]:
import re
from nltk.corpus import stopwords

In [214]:
### Dataset Preprocessing step
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Initialize an empty list to store the cleaned and stemmed text
corpus = []

# Loop through each message in the dataset
for i in range(0, len(messages)):
    # Ensure the title is a string, handle non-string values like NaN
    title = messages['title'][i]
    if not isinstance(title, str):
        continue  # Skip non-string entries
    
    # Remove non-alphabetic characters and convert to lowercase
    review = re.sub('[^a-zA-Z]', ' ', title)
    review = review.lower()
    review = review.split()
    
    # Remove stopwords and stem the words
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    
    # Join the words back into a single string
    review = ' '.join(review)
    
    # Append the processed review to the corpus
    corpus.append(review)

In [215]:
#Onehot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [216]:
#Embedding representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3697 4876 4062]
 [   0    0    0 ... 4685 3545  959]
 [   0    0    0 ... 2332 2505  970]
 ...
 [   0    0    0 ...  925 4672 4689]
 [   0    0    0 ... 4318 4047 4175]
 [   0    0    0 ... 3415 4694 1834]]


In [217]:
len(embedded_docs),y.shape

(18285, (18285,))

In [218]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [219]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [220]:
#Dropout
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

2024-07-31 17:14:32.243915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 17:14:32.244602: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 17:14:32.245584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [221]:
### Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


2024-07-31 17:14:32.394591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 17:14:32.395108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 17:14:32.395670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

189/192 [============================>.] - ETA: 0s - loss: 0.2960 - accuracy: 0.8639

2024-07-31 17:14:36.145870: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 17:14:36.146355: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 17:14:36.146797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

192/192 [==============================] - 4s 18ms/step - loss: 0.2948 - accuracy: 0.8645 - val_loss: 0.1891 - val_accuracy: 0.9205
Epoch 2/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1308 - accuracy: 0.9486 - val_loss: 0.1969 - val_accuracy: 0.9241
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0937 - accuracy: 0.9667 - val_loss: 0.2536 - val_accuracy: 0.9185
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0696 - accuracy: 0.9766 - val_loss: 0.2662 - val_accuracy: 0.9190
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0430 - accuracy: 0.9844 - val_loss: 0.2959 - val_accuracy: 0.9128
Epoch 6/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0261 - accuracy: 0.9920 - val_loss: 0.3490 - val_accuracy: 0.9133
Epoch 7/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0169 - accuracy: 0.9939 - val_loss: 0.3657 - val_accuracy: 0.9064
Epoch 8/10

In [222]:
#Performance

In [223]:
y_pred=model.predict(X_test)

 48/189 [======>.......................] - ETA: 0s

2024-07-31 17:15:06.755671: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-31 17:15:06.756316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-31 17:15:06.759076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

189/189 [==============================] - 1s 3ms/step


In [224]:
y_pred_final = (y_pred > 0.5)

In [225]:
from sklearn.metrics import confusion_matrix

In [226]:
confusion_matrix(y_test,y_pred_final)

array([[3122,  297],
       [ 253, 2363]])

In [227]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_final)

0.9088649544324772